In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
df = pd.read_csv('all_text_wakatigaki.csv',encoding='utf-8')
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


## TF値算出

In [3]:
# 単語名は同一でも品詞名が違う組み合わせがあるので単語名と品詞名を組み合わせた要素を格納した新規カラムを作成する
df['original_type'] = df['original'] + '_' + df['type']
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容,original_type
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般


In [4]:
# 必要カラムのみ抽出
df = df[['yomi', '内容', 'original_type']]
df.head()

,yomi,内容,original_type
0,ネオログディー,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般
1,_,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般
2,タガー,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般
3,.,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般
4,パース,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般


In [5]:
# [内容]×[original_type]の組み合わせの数をカウント
contents_word_cnt_df = df.groupby(['内容', 'original_type']).count().reset_index()
contents_word_cnt_df = contents_word_cnt_df.rename(columns={'yomi':'count'})
contents_word_cnt_df.head()

,内容,original_type,count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1


In [6]:
# 文書毎に含まれる単語数をカウント

# [内容]カラムをグループ化しカウント　
# reset_index関数を使い[内容]カラムをindex部分からデータ列部分に戻す
per_sentence_word_cnt = df.groupby('内容')['original_type'].count().reset_index()
per_sentence_word_cnt = per_sentence_word_cnt.rename(columns={'original_type':'word_count'})
per_sentence_word_cnt.head()

,内容,word_count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,26
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,30
2,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,33
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,41


In [7]:
# [count]と[word_count]を [内容]で紐づけ
contents_merge_df = pd.merge(contents_word_cnt_df, per_sentence_word_cnt, on='内容', how='inner')
contents_merge_df.head(27)

,内容,original_type,count,word_count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26
6,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般,1,26
7,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般,1,26
8,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,text_名詞-固有名詞-一般,1,26
9,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,、_記号-読点,1,26


In [8]:
# 欠損値があるか確認
contents_merge_df.isnull().any()

内容               False
original_type    False
count            False
word_count       False
dtype: bool

In [9]:
# tf値算出に必要なカラム（分子と分母の値があるカラム）のみ抽出
tmp_tf_df = contents_merge_df[['count', 'word_count']]
tmp_tf_df.head()

,count,word_count
0,1,26
1,1,26
2,1,26
3,1,26
4,1,26


In [10]:
# tf値算出
# axis=1をつけないと"occurred at index count"とエラーが発生する（デフォルトがaxis=0のため）
contents_merge_df['tf'] = tmp_tf_df.progress_apply(lambda x: x['count'] / x['word_count'], axis=1)

In [11]:
# tf値が紐づいたか確認
contents_merge_df.head()

,内容,original_type,count,word_count,tf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462


In [12]:
# csvファイルで出力
contents_merge_df.to_csv('contents_merge_df.csv', index=False, encoding='utf-8')

Mac環境の場合、csvファイルを出力した後

一旦CSVファイルを右クリックし、「このアプリケーションで開く」→「テキストエディット」を選択

command + shift + S をタップし、ファイルを複製

command + S をタップし保存画面を開く

「名前」を、任意の名前に「.csv」を付けた名前にして、「標準テキストのエンコーディング」を「日本語（Shift JIS）」として、保存

参考URL：https://support.freee.co.jp/hc/ja/articles/115002727126-CSV%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%8C%E6%96%87%E5%AD%97%E5%8C%96%E3%81%91%E3%81%97%E3%81%9F%E5%A0%B4%E5%90%88%E3%81%AE%E5%AF%BE%E5%87%A6%E6%96%B9%E6%B3%95#mac

In [13]:
# shift-jisのcsvファイル読み込み
contents_merge_df = pd.read_csv('contents_merge_df.csv', encoding='shift-jis')
contents_merge_df.head()

,内容,original_type,count,word_count,tf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462
